In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models
from keras.layers import Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
import datetime
import time

In [ ]:
!gsutil cp gs://gtsrb_raw_final/GTSRB_Final_Training_Images.zip .
!unzip GTSRB_Final_Training_Images.zip

In [3]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rescale=1./255,
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='training')

val_generator = datagen.flow_from_directory(
        'GTSRB/Final_Training/Images',
        target_size=(100, 100),
        batch_size=64,
        class_mode='categorical',
        subset='validation')

Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [4]:
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(100, 100, 3),
    classifier_activation="softmax",
)

2022-05-08 14:20:22.554988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 14:20:22.565790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 14:20:22.566400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-08 14:20:22.567580: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
model = models.Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=43, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dropout (Dropout)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 43)                1409067   
                                                                 
Total params: 24,996,779
Trainable params: 24,943,659
Non-trainable params: 53,120
_________________________________________________________________


In [7]:
%load_ext tensorboard

In [8]:
log_dir = "logs_resnet/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
steps_per_epoch = 729

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [12]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [13]:
tic = time.time()
model.fit(train_generator, validation_data=val_generator, epochs = 20, callbacks=[es,tensorboard_callback])
toc = time.time()

Epoch 1/20
491/491 [==============================] - 108s 220ms/step - loss: 0.0025 - accuracy: 0.9904 - val_loss: 0.0076 - val_accuracy: 0.9565
Epoch 2/20
491/491 [==============================] - 111s 226ms/step - loss: 0.0020 - accuracy: 0.9917 - val_loss: 0.0077 - val_accuracy: 0.9603
Epoch 3/20
491/491 [==============================] - 113s 229ms/step - loss: 0.0016 - accuracy: 0.9929 - val_loss: 0.0952 - val_accuracy: 0.8541
Epoch 4/20
491/491 [==============================] - 113s 229ms/step - loss: 0.0039 - accuracy: 0.9856 - val_loss: 0.0094 - val_accuracy: 0.9555
Epoch 5/20
491/491 [==============================] - 113s 230ms/step - loss: 0.0012 - accuracy: 0.9947 - val_loss: 0.0100 - val_accuracy: 0.9477
Epoch 6/20
491/491 [==============================] - 114s 231ms/step - loss: 0.0015 - accuracy: 0.9930 - val_loss: 0.0080 - val_accuracy: 0.9594
Epoch 7/20
491/491 [==============================] - 113s 230ms/step - loss: 6.1874e-04 - accuracy: 0.9970 - val_loss: 0.00

In [14]:
print('Time taken for training: {} '.format(toc-tic))

Time taken for training: 2257.881364583969 


In [ ]:
%tensorboard --logdir logs_inc_resnet/fit

In [98]:
for layer in resnet.layers:
    layer.trainable = False
model = models.Sequential()
model.add(resnet)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(units=43, activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 flatten_7 (Flatten)         (None, 32768)             0         
                                                                 
 dropout_7 (Dropout)         (None, 32768)             0         
                                                                 
 dense_7 (Dense)             (None, 43)                1409067   
                                                                 
Total params: 24,996,779
Trainable params: 1,409,067
Non-trainable params: 23,587,712
_________________________________________________________________


In [99]:
log_dir = "logs_resnet_NT/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [100]:
steps_per_epoch = 729

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

In [101]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [102]:
tic = time.time()
model.fit(train_generator, validation_data=val_generator, epochs = 10, callbacks=[es,tensorboard_callback])
toc = time.time()

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/10
491/491 [==============================] - 95s 189ms/step - loss: 0.0497 - accuracy: 0.8968 - val_loss: 0.0423 - val_accuracy: 0.8778
Epoch 2/10
491/491 [==============================] - 92s 187ms/step - loss: 0.0235 - accuracy: 0.9572 - val_loss: 0.0373 - val_accuracy: 0.8933
Epoch 3/10
491/491 [==============================] - 92s 187ms/step - loss: 0.0211 - accuracy: 0.9646 - val_loss: 0.0413 - val_accuracy: 0.8982
Epoch 4/10
491/491 [==============================] - 90s 183ms/step - loss: 0.0239 - accuracy: 0.9676 - val_loss: 0.0384 - val_accuracy: 0.9015
Epoch 5/10
491/491 [==============================] - 90s 183ms/step - loss: 0.0226 - accuracy: 0.9699 - val_loss: 0.0344 - val_accuracy: 0.9037
Epoch 6/10
491/491 [==============================] - 91s 185ms/step - loss: 0.0210 - accuracy: 0.9712 - val_loss: 0.0529 - val_accuracy: 0.9079
Epoch 7/10
491/491 [==============================] - 90s 183ms/step - loss: 0.0234 - accuracy: 0.9711 - val_loss: 0.0378 - val_ac

In [103]:
print('Time taken for training: {} '.format(toc-tic))

Time taken for training: 908.985512971878 


In [ ]:
%tensorboard --logdir logs_resnet_NT/fit